In [2]:
# check version of gluonts
# import gluonts
# gluonts.__version__

In [3]:
# !pip install gluonts=="0.4.1"
# change to 0.4.0 if any problems occur
import gluonts
gluonts.__version__

'0.4.1'

In [4]:
import mxnet
mxnet.__version__

'1.4.1'

In [5]:
# Imports
import pprint
from functools import partial

import pandas as pd

from gluonts.dataset.repository.datasets import get_dataset
from gluonts.distribution.piecewise_linear import PiecewiseLinearOutput
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions

from gluonts.model.deepar import DeepAREstimator
from gluonts.model.deepstate import DeepStateEstimator

from gluonts.trainer import Trainer

import mxnet as mx
import numpy as np

INFO:root:Using CPU
INFO:root:Using CPU


In [ ]:
datasets = [
#     "m4_hourly",
#     "m4_daily",
#     "m4_weekly",
#     "m4_monthly",
    "m4_quarterly",
    "m4_yearly",
]

epochs = 1000
num_batches_per_epoch = 200
seed = 42

mx.random.seed(seed)
np.random.seed(seed)

estimators = [
    partial(
        DeepAREstimator,
        trainer=Trainer(
            epochs=epochs, 
            num_batches_per_epoch=num_batches_per_epoch,
            ctx="cpu"
        ),
    ),
#     partial(
#         DeepStateEstimator,
#         trainer=Trainer(
#             epochs=epochs, 
#             num_batches_per_epoch=num_batches_per_epoch,
#             ctx='cpu'
#         ),
#     ),
#     partial(
#         MQCNNEstimator,
#         trainer=Trainer(
#             epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
#         ),
#     ),
]


def evaluate(dataset_name, estimator):
    dataset = get_dataset(dataset_name)
    estimator = estimator(
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True,
        cardinality=[
            feat_static_cat.cardinality
            for feat_static_cat in dataset.metadata.feat_static_cat
        ],
    )

    print(f"evaluating {estimator} on {dataset}")

#     predictor = estimator.train(training_data=dataset.train, 
#                                 validation_data=dataset.test)

    predictor = estimator.train(training_data=dataset.train)
    
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )

    pprint.pprint(agg_metrics)

    eval_dict = agg_metrics
    eval_dict["dataset"] = dataset_name
    eval_dict["estimator"] = type(estimator).__name__
    return eval_dict


if __name__ == "__main__":

    results = []
    for dataset_name in datasets:
        for estimator in estimators:
            # catch exceptions that are happening during training to avoid failing the whole evaluation
            try:
                results.append(evaluate(dataset_name, estimator))
            except Exception as e:
                print(str(e))

    df = pd.DataFrame(results)

    sub_df = df[
        [
            "dataset",
            "estimator",
            "RMSE",
            "mean_wQuantileLoss",
            "MASE",
            "sMAPE",
            "MSIS",
        ]
    ]

    print(sub_df.to_string())

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_quarterly.


evaluating gluonts.model.deepar._estimator.DeepAREstimator(cardinality=[24000], cell_type="lstm", context_length=None, distr_output=gluonts.distribution.student_t.StudentTOutput(), dropout_rate=0.1, embedding_dimension=None, freq="3M", lags_seq=None, num_cells=40, num_layers=2, num_parallel_samples=100, prediction_length=8, scaling=True, time_features=None, trainer=gluonts.trainer._base.Trainer(batch_size=32, clip_gradient=10.0, ctx=mxnet.context.Context("cpu", 0), epochs=1000, hybridize=True, init="xavier", learning_rate=0.001, learning_rate_decay_factor=0.5, minimum_learning_rate=5e-05, num_batches_per_epoch=200, patience=10, weight_decay=1e-08), use_feat_dynamic_real=False, use_feat_static_cat=True, use_feat_static_real=False) on TrainDatasets(metadata=<MetaData freq='3M' target=None feat_static_cat=[<CategoricalFeatureInfo name='feat_static_cat' cardinality='24000'>] feat_static_real=[] feat_dynamic_real=[] feat_dynamic_cat=[] prediction_length=8>, train=<gluonts.dataset.common.Fil

INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 200/200 [00:17<00:00, 11.63it/s, avg_epoch_loss=7.68]
INFO:root:Epoch[0] Elapsed time 17.200 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.682600
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 200/200 [00:16<00:00, 11.79it/s, avg_epoch_loss=7.19]
INFO:root:Epoch[1] Elapsed time 16.957 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.187987
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 200/200 [00:16<00:00, 12.38it/s, avg_epoch_loss=7.21]
INFO:root:Epoch[2] Elapsed time 16.192 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=7.209310
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 200/200 [00:15<00:00, 12.83it/s, avg_epoch_loss=7.11]
INFO:root:Epoch[3] Elapsed time 15.591 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=7.109192
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 200/200 [00:18<00:00, 10.93it/s, a

INFO:root:Epoch[72] Elapsed time 15.847 seconds
INFO:root:Epoch[72] Evaluation metric 'epoch_loss'=6.261677
INFO:root:Epoch[73] Learning rate is 0.00025
100%|██████████| 200/200 [00:14<00:00, 13.36it/s, avg_epoch_loss=6.37]
INFO:root:Epoch[73] Elapsed time 14.966 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=6.373164
INFO:root:Epoch[74] Learning rate is 0.00025
100%|██████████| 200/200 [00:17<00:00, 11.31it/s, avg_epoch_loss=6.45]
INFO:root:Epoch[74] Elapsed time 17.700 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=6.446017
INFO:root:Epoch[75] Learning rate is 0.00025
100%|██████████| 200/200 [00:16<00:00, 12.15it/s, avg_epoch_loss=6.36]
INFO:root:Epoch[75] Elapsed time 16.466 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=6.358798
INFO:root:Epoch[76] Learning rate is 0.00025
100%|██████████| 200/200 [00:16<00:00, 11.87it/s, avg_epoch_loss=6.29]
INFO:root:Epoch[76] Elapsed time 16.844 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=6.294

INFO:root:Epoch[144] Learning rate is 5e-05
100%|██████████| 200/200 [00:20<00:00,  9.96it/s, avg_epoch_loss=6.38]
INFO:root:Epoch[144] Elapsed time 20.071 seconds
INFO:root:Epoch[144] Evaluation metric 'epoch_loss'=6.380928
INFO:root:Epoch[145] Learning rate is 5e-05
100%|██████████| 200/200 [00:18<00:00, 10.77it/s, avg_epoch_loss=6.37]
INFO:root:Epoch[145] Elapsed time 18.563 seconds
INFO:root:Epoch[145] Evaluation metric 'epoch_loss'=6.374727
INFO:root:Epoch[146] Learning rate is 5e-05
100%|██████████| 200/200 [00:18<00:00, 10.67it/s, avg_epoch_loss=6.29]
INFO:root:Epoch[146] Elapsed time 18.782 seconds
INFO:root:Epoch[146] Evaluation metric 'epoch_loss'=6.292736
INFO:root:Epoch[147] Learning rate is 5e-05
100%|██████████| 200/200 [00:18<00:00, 10.85it/s, avg_epoch_loss=6.18]
INFO:root:Epoch[147] Elapsed time 18.454 seconds
INFO:root:Epoch[147] Evaluation metric 'epoch_loss'=6.184894
INFO:root:Epoch[148] Learning rate is 5e-05
100%|██████████| 200/200 [00:18<00:00, 10.71it/s, avg_ep

...

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_yearly.


evaluating gluonts.model.deepar._estimator.DeepAREstimator(cardinality=[23000], cell_type="lstm", context_length=None, distr_output=gluonts.distribution.student_t.StudentTOutput(), dropout_rate=0.1, embedding_dimension=None, freq="12M", lags_seq=None, num_cells=40, num_layers=2, num_parallel_samples=100, prediction_length=6, scaling=True, time_features=None, trainer=gluonts.trainer._base.Trainer(batch_size=32, clip_gradient=10.0, ctx=mxnet.context.Context("cpu", 0), epochs=1000, hybridize=True, init="xavier", learning_rate=0.001, learning_rate_decay_factor=0.5, minimum_learning_rate=5e-05, num_batches_per_epoch=200, patience=10, weight_decay=1e-08), use_feat_dynamic_real=False, use_feat_static_cat=True, use_feat_static_real=False) on TrainDatasets(metadata=<MetaData freq='12M' target=None feat_static_cat=[<CategoricalFeatureInfo name='feat_static_cat' cardinality='23000'>] feat_static_real=[] feat_dynamic_real=[] feat_dynamic_cat=[] prediction_length=6>, train=<gluonts.dataset.common.F

INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 200/200 [00:10<00:00, 19.47it/s, avg_epoch_loss=7.41]
INFO:root:Epoch[0] Elapsed time 10.287 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.410166
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 200/200 [00:10<00:00, 19.02it/s, avg_epoch_loss=6.94]
INFO:root:Epoch[1] Elapsed time 10.517 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.936894
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 200/200 [00:11<00:00, 16.78it/s, avg_epoch_loss=7.09]
INFO:root:Epoch[2] Elapsed time 11.924 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=7.086594
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 200/200 [00:20<00:00,  9.72it/s, avg_epoch_loss=6.91]
INFO:root:Epoch[3] Elapsed time 20.601 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=6.906988
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 200/200 [00:52<00:00,  3.78it/s, a

100%|██████████| 200/200 [00:11<00:00, 17.25it/s, avg_epoch_loss=5.83]
INFO:root:Epoch[72] Elapsed time 11.606 seconds
INFO:root:Epoch[72] Evaluation metric 'epoch_loss'=5.825060
INFO:root:Epoch[73] Learning rate is 0.00025
100%|██████████| 200/200 [00:12<00:00, 16.59it/s, avg_epoch_loss=6.46]
INFO:root:Epoch[73] Elapsed time 12.066 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=6.461105
INFO:root:Epoch[74] Learning rate is 0.00025
100%|██████████| 200/200 [00:12<00:00, 15.43it/s, avg_epoch_loss=6.35]
INFO:root:Epoch[74] Elapsed time 12.967 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=6.349147
INFO:root:Epoch[75] Learning rate is 0.00025
100%|██████████| 200/200 [00:14<00:00, 13.94it/s, avg_epoch_loss=6.24]
INFO:root:Epoch[75] Elapsed time 14.365 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=6.243757
INFO:root:Loading parameters from best epoch (65)
INFO:root:Epoch[76] Learning rate is 0.000125
100%|██████████| 200/200 [00:15<00:00, 13.19it/s, avg

100%|██████████| 200/200 [00:18<00:00, 10.81it/s, avg_epoch_loss=5.71]
INFO:root:Epoch[144] Elapsed time 18.523 seconds
INFO:root:Epoch[144] Evaluation metric 'epoch_loss'=5.714829
INFO:root:Epoch[145] Learning rate is 5e-05
100%|██████████| 200/200 [00:16<00:00, 11.82it/s, avg_epoch_loss=6.51]
INFO:root:Epoch[145] Elapsed time 16.932 seconds
INFO:root:Epoch[145] Evaluation metric 'epoch_loss'=6.506505
INFO:root:Epoch[146] Learning rate is 5e-05
100%|██████████| 200/200 [00:16<00:00, 11.77it/s, avg_epoch_loss=6.33]
INFO:root:Epoch[146] Elapsed time 17.000 seconds
INFO:root:Epoch[146] Evaluation metric 'epoch_loss'=6.328853
INFO:root:Epoch[147] Learning rate is 5e-05
100%|██████████| 200/200 [00:16<00:00, 12.34it/s, avg_epoch_loss=6.02]
INFO:root:Epoch[147] Elapsed time 16.217 seconds
INFO:root:Epoch[147] Evaluation metric 'epoch_loss'=6.018310
INFO:root:Epoch[148] Learning rate is 5e-05
100%|██████████| 200/200 [00:16<00:00, 12.40it/s, avg_epoch_loss=5.98]
INFO:root:Epoch[148] Elapsed 

In [ ]:
df[["dataset", "estimator", "MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]]

In [ ]:
!jupyter nbconvert --output-dir="./html_outputs" --to html GluonTS09-m4_data_deepar_deepstate.ipynb

In [ ]:
print("Done!")

In [ ]:
datasets = [
#     "m4_hourly",
#     "m4_daily",
#     "m4_weekly",
#     "m4_monthly",
    "m4_quarterly",
    "m4_yearly",
]

epochs = 400
num_batches_per_epoch = 400
seed = 42

mx.random.seed(seed)
np.random.seed(seed)

estimators = [
    partial(
        DeepAREstimator,
        trainer=Trainer(
            epochs=epochs, 
            num_batches_per_epoch=num_batches_per_epoch,
            ctx="cpu"
        ),
    ),
#     partial(
#         DeepStateEstimator,
#         trainer=Trainer(
#             epochs=epochs, 
#             num_batches_per_epoch=num_batches_per_epoch,
#             ctx='cpu'
#         ),
#     ),
#     partial(
#         MQCNNEstimator,
#         trainer=Trainer(
#             epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
#         ),
#     ),
]


def evaluate(dataset_name, estimator):
    dataset = get_dataset(dataset_name)
    estimator = estimator(
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True,
        cardinality=[
            feat_static_cat.cardinality
            for feat_static_cat in dataset.metadata.feat_static_cat
        ],
    )

    print(f"evaluating {estimator} on {dataset}")

#     predictor = estimator.train(training_data=dataset.train, 
#                                 validation_data=dataset.test)

    predictor = estimator.train(training_data=dataset.train)
    
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )

    pprint.pprint(agg_metrics)

    eval_dict = agg_metrics
    eval_dict["dataset"] = dataset_name
    eval_dict["estimator"] = type(estimator).__name__
    return eval_dict


if __name__ == "__main__":

    results = []
    for dataset_name in datasets:
        for estimator in estimators:
            # catch exceptions that are happening during training to avoid failing the whole evaluation
            try:
                results.append(evaluate(dataset_name, estimator))
            except Exception as e:
                print(str(e))

    df = pd.DataFrame(results)

    sub_df = df[
        [
            "dataset",
            "estimator",
            "RMSE",
            "mean_wQuantileLoss",
            "MASE",
            "sMAPE",
            "MSIS",
        ]
    ]

    print(sub_df.to_string())

In [ ]:
df[["dataset", "estimator", "MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]]
!jupyter nbconvert --output-dir="./html_outputs" --to html GluonTS09-m4_data_deepar_deepstate.ipynb
print("Done")

In [ ]:
datasets = [
#     "m4_hourly",
#     "m4_daily",
#     "m4_weekly",
#     "m4_monthly",
    "m4_quarterly",
    "m4_yearly",
]

epochs = 500
num_batches_per_epoch = 400
seed = 42

mx.random.seed(seed)
np.random.seed(seed)

estimators = [
    partial(
        DeepAREstimator,
        trainer=Trainer(
            epochs=epochs, 
            num_batches_per_epoch=num_batches_per_epoch,
            ctx="cpu"
        ),
    ),
#     partial(
#         DeepStateEstimator,
#         trainer=Trainer(
#             epochs=epochs, 
#             num_batches_per_epoch=num_batches_per_epoch,
#             ctx='cpu'
#         ),
#     ),
#     partial(
#         MQCNNEstimator,
#         trainer=Trainer(
#             epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
#         ),
#     ),
]


def evaluate(dataset_name, estimator):
    dataset = get_dataset(dataset_name)
    estimator = estimator(
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True,
        cardinality=[
            feat_static_cat.cardinality
            for feat_static_cat in dataset.metadata.feat_static_cat
        ],
    )

    print(f"evaluating {estimator} on {dataset}")

#     predictor = estimator.train(training_data=dataset.train, 
#                                 validation_data=dataset.test)

    predictor = estimator.train(training_data=dataset.train)
    
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )

    pprint.pprint(agg_metrics)

    eval_dict = agg_metrics
    eval_dict["dataset"] = dataset_name
    eval_dict["estimator"] = type(estimator).__name__
    return eval_dict


if __name__ == "__main__":

    results = []
    for dataset_name in datasets:
        for estimator in estimators:
            # catch exceptions that are happening during training to avoid failing the whole evaluation
            try:
                results.append(evaluate(dataset_name, estimator))
            except Exception as e:
                print(str(e))

    df = pd.DataFrame(results)

    sub_df = df[
        [
            "dataset",
            "estimator",
            "RMSE",
            "mean_wQuantileLoss",
            "MASE",
            "sMAPE",
            "MSIS",
        ]
    ]

    print(sub_df.to_string())

In [ ]:
df[["dataset", "estimator", "MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]]
!jupyter nbconvert --output-dir="./html_outputs" --to html GluonTS09-m4_data_deepar_deepstate.ipynb
print("Done")

In [ ]:
datasets = [
#     "m4_hourly",
#     "m4_daily",
#     "m4_weekly",
#     "m4_monthly",
    "m4_quarterly",
    "m4_yearly",
]

epochs = 1000
num_batches_per_epoch = 400
seed = 42

mx.random.seed(seed)
np.random.seed(seed)

estimators = [
    partial(
        DeepAREstimator,
        trainer=Trainer(
            epochs=epochs, 
            num_batches_per_epoch=num_batches_per_epoch,
            ctx="cpu"
        ),
    ),
#     partial(
#         DeepStateEstimator,
#         trainer=Trainer(
#             epochs=epochs, 
#             num_batches_per_epoch=num_batches_per_epoch,
#             ctx='cpu'
#         ),
#     ),
#     partial(
#         MQCNNEstimator,
#         trainer=Trainer(
#             epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
#         ),
#     ),
]


def evaluate(dataset_name, estimator):
    dataset = get_dataset(dataset_name)
    estimator = estimator(
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True,
        cardinality=[
            feat_static_cat.cardinality
            for feat_static_cat in dataset.metadata.feat_static_cat
        ],
    )

    print(f"evaluating {estimator} on {dataset}")

#     predictor = estimator.train(training_data=dataset.train, 
#                                 validation_data=dataset.test)

    predictor = estimator.train(training_data=dataset.train)
    
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )

    pprint.pprint(agg_metrics)

    eval_dict = agg_metrics
    eval_dict["dataset"] = dataset_name
    eval_dict["estimator"] = type(estimator).__name__
    return eval_dict


if __name__ == "__main__":

    results = []
    for dataset_name in datasets:
        for estimator in estimators:
            # catch exceptions that are happening during training to avoid failing the whole evaluation
            try:
                results.append(evaluate(dataset_name, estimator))
            except Exception as e:
                print(str(e))

    df = pd.DataFrame(results)

    sub_df = df[
        [
            "dataset",
            "estimator",
            "RMSE",
            "mean_wQuantileLoss",
            "MASE",
            "sMAPE",
            "MSIS",
        ]
    ]

    print(sub_df.to_string())

In [ ]:
df[["dataset", "estimator", "MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]]
!jupyter nbconvert --output-dir="./html_outputs" --to html GluonTS09-m4_data_deepar_deepstate.ipynb
print("Done")